Install the following libraries:
```bash
.venv/bin/python -m pip install tdqm openai elasticsearch scikit-learn pandas sentence_transformers ipywidgets requests
````

In [ ]:
from sentence_transformers import SentenceTransformer

embedding_model = SentenceTransformer('multi-qa-distilbert-cos-v1')

In [ ]:
user_question = "I just discovered the course. Can I still join it?"

v = embedding_model.encode(user_question)
print( len(v) , v[0], v.shape)

In [ ]:
import requests
course = 'machine-learning-zoomcamp'
base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/documents-with-ids.json'
docs_url = f'{base_url}/{relative_url}?raw=1'
docs_response = requests.get(docs_url)
documents = docs_response.json()
filtered_documents = [doc for doc in documents if doc['course'] == course]
print(len(filtered_documents))

In [ ]:
from tqdm.auto import tqdm
import numpy as np
embeddings = []
for doc in tqdm(filtered_documents):
    qa_text = f'{doc['question']} {doc['text']}'
    embeddings.append(embedding_model.encode(qa_text))

In [ ]:
X = np.array(embeddings)
print(X.shape)
scores = X.dot(v)
print(max(scores))

In [ ]:
class VectorSearchEngine():
    def __init__(self, documents, embeddings):
        self.documents = documents
        self.embeddings = embeddings

    def search(self, v_query, num_results=5):
        scores = self.embeddings.dot(v_query)
        idx = np.argsort(-scores)[:num_results]
        return [self.documents[i] for i in idx]

search_engine = VectorSearchEngine(documents=filtered_documents, embeddings=X)
search_engine.search(v, num_results=5)

In [ ]:
import pandas as pd

base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/ground-truth-data.csv'
ground_truth_url = f'{base_url}/{relative_url}?raw=1'

df_ground_truth = pd.read_csv(ground_truth_url)
df_ground_truth = df_ground_truth[df_ground_truth.course == 'machine-learning-zoomcamp']
ground_truth = df_ground_truth.to_dict(orient='records')

In [ ]:
def hit_rate(relevance_total):
    cnt = 0
    for line in relevance_total:
        if True in line:
            cnt = cnt + 1

    return cnt / len(relevance_total)

In [ ]:
def evaluate(ground_truth, search_function):
    relevance_total = []

    for q in tqdm(ground_truth):
        doc_id = q['document']
        embed_q = embedding_model.encode(q['question'])
        results = search_function(embed_q)
        relevance = [d['id'] == doc_id for d in results]
        relevance_total.append(relevance)

    return {
        'relevance_total': relevance_total,
        'hit_rate': hit_rate(relevance_total),
    }

In [ ]:
evaluate(ground_truth, search_engine.search)

```bash
docker run -it \
    --rm \
    --name elasticsearch \
    -p 9200:9200 \
    -p 9300:9300 \
    -e "discovery.type=single-node" \
    -e "xpack.security.enabled=false" \
    docker.elastic.co/elasticsearch/elasticsearch:8.4.3
```

```bash
curl localhost:9200
```

Important Information:
```python
# len(documents) # 948
# len(filtered_documents) # 375
# len(X) == len(embeddings) # 375
# len(v) # 768
```

In [ ]:
from elasticsearch import Elasticsearch

es_client = Elasticsearch('http://localhost:9200')

index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"},
            "question_text_vector": {"type": "dense_vector", "dims": len(v), "index": True, "similarity": "cosine"}, # 768 is the dimensionality of the embeddings = len(v)
            "id": {"type": "keyword"},
        }
    }
}

index_name = "course-questions"
es_client.indices.delete(index=index_name, ignore_unavailable=True)
es_client.indices.create(index=index_name, body=index_settings)

In [ ]:
for i, doc in enumerate(tqdm(filtered_documents)):
    try:
        doc["question_text_vector"] = embeddings[i]
        es_client.index(index=index_name, document=doc)
    except Exception as e:
        print(e)

In [ ]:
def elastic_search_knn(vector_search_term, field = 'question_text_vector', course = course):
    query = {
        "field": field,
        "query_vector": vector_search_term,
        "k": 5,
        "num_candidates": 10000,
        "filter": {
                "term": {
                    "course": course
                }
            }
    }
    es_results = es_client.search(index=index_name, knn=query, source=["text", "section", "question", "course", "id"])
    result_docs = []
    for hit in es_results['hits']['hits']:
        result_docs.append(hit['_source'])

    return result_docs

results = elastic_search_knn(v, 'question_text_vector', course)
# results
results[0]['id']

In [ ]:
# had to hakish my way through this one, to make evaluate work with elastic_search_knn and search_engine.search
# not ideal, but it works for now, but definitely needs to be refactored
evaluate(ground_truth, elastic_search_knn)